### A Simple NN

In [ ]:
## Importing Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
## Model
class CNN(nn.Module):
  def __init__(self,in_channels=1,num_classes = 10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), padding=(1,1)) # Same Convolution
    self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), padding=(1,1)) # Same Convolution
    self.fc1 = nn.Linear(16*7*7,num_classes) # Will use 2 maxpool

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)

    return x


In [ ]:
## Test model
model = CNN()
x = torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
## Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [ ]:
## Load Data
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
## Initialize Network
model = CNN().to(device)

In [ ]:
## Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
## Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores,targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or Adam step size
    optimizer.step()

100%|██████████| 938/938 [00:07<00:00, 124.36it/s]


In [ ]:
## Check Accuracy on training & test to see how good our model 
def check_accuracy(loader,model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y  in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      # Forward
      scores = model(x)
      #64x10
      _,predictions = scores.max(1) #(max values for 2nd dim)/will output indices
      
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0) #Size of 1st dimension

  model.train()
  return num_correct/num_samples

print(f'Accuracy on training data: {check_accuracy(train_loader, model)*100:.2f}')
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training data: 98.59
Accuracy on test set: 98.51
